In [1]:
import os
import xlwt
import xlrd
import sys
import numpy as np

In [2]:
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'  
os.environ['HTTPS_PROXY'] = 'https://127.0.0.1:7890'

import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VPcuIT5Chy-VkXnyqqa_xIaKXINJYjv783TZ91kGlmYXAP5G04cLFM

Successfully saved authorization token.


In [3]:
def GetBounds(point,radius,proj):
    radius=radius-5
#     X=ee.Number(point.transform("EPSG:32611",0.5).coordinates().get(0))
    X=ee.Number(point.coordinates().get(0))
#     print('X',X)
#     Y=ee.Number(point.transform("EPSG:32611",0.5).coordinates().get(1))
    Y=ee.Number(point.coordinates().get(1))
#     print('Y',Y)
    X_min=X.subtract(radius)  
#     print('X_min=',X_min)
    X_max=X.add(radius)
#     print('X_max=',X_max)
    Y_min=Y.subtract(radius)
#     print('Y_min=',Y_min)
    Y_max=Y.add(radius)
#     print('Y_max=',Y_max)
    Bound=ee.Geometry.Rectangle(coords=[X_min,Y_min,X_max,Y_max], proj=proj, geodesic=False, evenOdd=True)
    return Bound

def SetBounds(feature,width,resolution):
    return feature.setGeometry(GetBounds(feature.geometry(),width/2*resolution)) 

In [52]:

# Wa
# point_path = r"D:\study_Irrigation\shp\Wa_sample\point_utm\point_train.xls"
# point_path = r"D:\study_Irrigation\shp\Wa_sample\point_utm\point_val.xls"
# Ca
point_path = r"D:\study_Irrigation\shp\Ca_sample\point_utm\point_train.xls"
# point_path = r"D:\study_Irrigation\shp\Ca_sample\point_utm\point_val.xls"

proj_point = "EPSG:32611"

image_type = 'COPERNICUS/S2_HARMONIZED'

resolution=10

# state = "Wa"
state = "Ca"

month = 10

# Wa
# if month in [1,3,5,7,8]:   
#     StartDate=ee.Date('2020-0{}-01'.format(month)); 
#     EndDate=ee.Date('2020-0{}-31'.format(month));  
# elif month in [10,12]:   
#     StartDate=ee.Date('2020-{}-01'.format(month));  
#     EndDate=ee.Date('2020-{}-31'.format(month));  
# elif month in [4,6,9]:   
#     StartDate=ee.Date('2020-0{}-01'.format(month));
#     EndDate=ee.Date('2020-0{}-30'.format(month));  
# elif month in [11]:   
#     StartDate=ee.Date('2020-{}-01'.format(month));
#     EndDate=ee.Date('2020-{}-30'.format(month)); 

# Ca
if month in [1,3,5,7,8]:   
    StartDate=ee.Date('2019-0{}-01'.format(month)); 
    EndDate=ee.Date('2019-0{}-31'.format(month));  
elif month in [10,12]:   
    StartDate=ee.Date('2019-{}-01'.format(month));  
    EndDate=ee.Date('2019-{}-31'.format(month));  
elif month in [4,6,9]:   
    StartDate=ee.Date('2019-0{}-01'.format(month));
    EndDate=ee.Date('2019-0{}-30'.format(month));  
elif month in [11]:   
    StartDate=ee.Date('2019-{}-01'.format(month));
    EndDate=ee.Date('2019-{}-30'.format(month)); 

cloud_percent = 20

folder = "{}_train_{}".format(state,month)
# folder = "{}_val_{}".format(state,month)

scale = 10

crs = "EPSG:32611"

In [53]:

data = xlrd.open_workbook(point_path) 
table = data.sheets()[0]   
nrows = table.nrows  

result=[]
for i in range(nrows):   
    if i == 0: 
        print(table.row_values(i))  
        continue

    result.append(table.row_values(i))
    if i == 1: 
        print(table.row_values(i))  # [1.0, 768.0, 96187.54838725406, 5435212.05258369]
        print(type(table.row_values(i)[0]))  

size=int(np.size(result)/4);
print(size)

['name_id', 'size', 'geoX', 'geoY']
[180.0, 256.0, 511333.9283844559, 3632659.3631517687]
<class 'float'>
207


In [54]:
for i in range(0,size):
    id = int(result[i][0])
    if id not in [199,200,201,202,203,204,205,206,207]:  
        continue  
    if id < 10:
        name_id = "00" + str(id) 
    elif 10 <= id < 100:
        name_id = "0" + str(id)
    else:
        name_id = str(id)
    width = int(result[i][1])
    print("{0}月---------第{1}个--------id_{2}---------裁剪尺寸{3}".format(month,i+1,name_id,width))
    point_X = float(result[i][2])
    point_Y = float(result[i][3])
    point = ee.Geometry.Point([point_X,point_Y],proj=proj_point)

    studyArea= GetBounds(point,width/2*resolution,proj_point)
#     print('studyArea',studyArea)
    S2 = ee.ImageCollection(image_type).filterBounds(studyArea).filterDate(StartDate,EndDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percent)).select(['B2','B3','B4','B8']).median();

#     print(S2.size().getInfo());

#     compImg_B2=S2.select(['B2']).reduce(ee.Reducer.percentile([25])).rename('blue');
#     compImg_B3=S2.select(['B3']).reduce(ee.Reducer.percentile([25])).rename('green');
#     compImg_B4=S2.select(['B4']).reduce(ee.Reducer.percentile([25])).rename('red');
#     compImg_B8=S2.select(['B8']).reduce(ee.Reducer.percentile([25])).rename('nir');

#     compImg=S2.clip(studyArea.transform("EPSG:32611",0.1))
    compImg=S2.clip(studyArea)

    export = ee.batch.Export.image.toDrive(
        image=compImg,
        description=folder+"_"+name_id,
        fileNamePrefix=folder+"_"+name_id,
        folder=folder,
        scale=scale,
        region=studyArea,
        maxPixels=1e13,
        crs=crs)
    export.start()

10月---------第199个--------id_199---------裁剪尺寸768
10月---------第200个--------id_200---------裁剪尺寸768
10月---------第201个--------id_201---------裁剪尺寸768
10月---------第202个--------id_202---------裁剪尺寸768
10月---------第203个--------id_203---------裁剪尺寸768
10月---------第204个--------id_204---------裁剪尺寸512
10月---------第205个--------id_205---------裁剪尺寸512
10月---------第206个--------id_206---------裁剪尺寸512
10月---------第207个--------id_207---------裁剪尺寸768
